## 1.在Python中是如何管理内存的？

答：

由于Cpython是使用最广泛的Python实现（解释和编译），因此本解答为Cpython的内存管理器机制。
较完整的内容可参考我的笔记：https://nbviewer.jupyter.org/github/xiaoniaoyouhuajiang/ClassNotebook/blob/main/interview/Python%E7%9A%84%E5%86%85%E5%AD%98%E7%AE%A1%E7%90%86%E6%9C%BA%E5%88%B6.ipynb

## 2.迭代器和生成器的区别

答：

迭代器一般指可迭代对象。从**迭代器协议**出发，迭代器协议指对象的实现中提供了**next特殊方法**，使得该函数能够迭代自身容器中的下一项或是引起StopIteration异常终止迭代。而可迭代对象等价于**实现了迭代器协议**的对象。Python内置的容器类数据结构，如list,dict等都实现了迭代器协议，因此属于可迭代对象。

而对于迭代器，我们可以用for循环,sum,min,max等函数对容器内的内容进行访问。

而生成器，就像是一个惰性的迭代器，实现了延迟操作，在需要的时候才产生结果，而不是立即产生结果。

Python中有两种方式可以生成一个生成器。
* 生成器函数：函数定义不变，只是使用yield替代return关键字，使得每次返回一个结果，并挂起函数状态。下次再进行调用则在上次离开时的地方继续。
* 生成器表达式：语法上类似于容器推导（如列表推到），但返回的结果是一个generator对象，并能被next()函数调用。

应用如：

```python
def get_primes():
    candidate = 2
    found = []
    while True:
        if all(candidate % prime != 0 for prime in found):
            yield candidate
            found.append(candidate)
        candidate += 1
        
primes = get_primes()
# <generator object get_primes at 0x00000223FE1B17C8>

next(primes), next(primes)
# (2, 3)
```

其他应用还包括对于大数据量的计算，如：

```python
sum(i for i in range(10000000000))
```
生成器是Python一个比较有特色的内容，其优点是保存了迭代的中间状态，可应对一些特殊场景。

## 3.使用lambda函数实现两数相乘
答：

```python
f = lambda x,y: x*y
f(2,4)
# 8
```

## 4.Python中的闭包是什么
答：

Python中函数是一等公民，能够被作为参数传递。另一方面，也可以实现高阶函数，而闭包指的就是函数嵌套中的内层函数，假设调用函数A，函数A中嵌套了函数B，这样函数B就可以引用函数A中域的变量，从而实现闭包的调用。这个特性非常有用，其体现的是将函数作为一种可调用对象的思想，下面给出一个我认为比较有用的例子：

```python
def compose(*funcs):
    """
    compose(f,g,...) == f(g...(x))
    """
    def inner(data, funcs = funcs):
        result = data
        for f in reversed(funcs):
            result = f(result)
        return result
    return inner
```

正如注释所言，通过inner逻辑调用*func，我们实现了compose(f,g,...) == f(g...(x))


## 5.django,flask,tornado框架比较？

* django

django大而全，自身提供了较齐全的工具链。包括ORM,template,甚至还有像Django Cache这样的缓存工具。相比于其他两个框架，其开发效率相对高，但相对而言，可定制性较差，很多时候是框架限制了开发。

* flask

flask在灵活性上的优势是最强的，同时也是轻量化的web框架，其源码的第一个release代码量（不包测试）只有500多行。另一方面，其路由设定使用装饰器，较方便快捷。比起django，flask耦合度较低，因为他的其他组件大多是自己开发或是来源于第三方的开发，有非常高的灵活度。但对于大型网站，其路由映射规则需要单独设模块设置，否则容易混乱。

* tornado

tornado的最大特点是其异步非阻塞的特性，这个特性应对IO密集型和多任务处理上有非常大的优势。tornado注重性能，由于其请求处理是基于回调的非阻塞调用，因此并发性能较好。同时内置了http服务器，其模型是单线程异步网络，会根据CPU数量运行多个实例。比起flask,django，tornado最大的优势就是在并发数较高时有比较好的表现。



## 6.谈谈对restful规范的认识

restful是一种http api的一种规范，其最初是一篇论文中提出的接口设计原则，现阶段人们讨论的restful已和最初论文中的思想不一致（作者本人也提到了），我认为restful体现的是4个web api接口设计的原则：

* C-S架构：数据存储在server,Client只需使用，两端彻底分离。
* 无状态：HTTP请求本身就是无状态的，每一次请求都必须带上所有信息，且两端之间不会记录任何状态。
* 统一的接口：简单来说，接口的设计必须有统一性。这样才能提高接口的可读性。
* 一致的数据格式： 服务端只返回三种信息：Json/XML/状态码。

其中接口的设计规范内容较多，此处不展开，可参考我的笔记：https://nbviewer.jupyter.org/github/xiaoniaoyouhuajiang/ClassNotebook/blob/main/web_api_design/API_design.ipynb

## 7.从在浏览器地址中输入域名到数据返回发生了什么？
发生了：

1.浏览器会完成地址解析，HSTS，检查缓存等操作，如解析没有异常，存在缓存则直接使用缓存则直接返回缓存，没有则将请求通过OS再通过网卡等硬件转化为电信号向远端发起请求。

2.紧接着是DNS查询，浏览器有DNS缓存，如果发现DNS不在缓存中，则交由OS，首先查看是否存在HOSTS文件中，没有则向本地DNS服务器发送查询请求，再又本地DNS服务器转发到多个互联网根域，最终获得IP地址。

3.获取到IP则要进行连接。注意，浏览器会将请求报文封装好后给远端服务器接收并解封装，这个封装与解封装过程为可逆过程。TCP/IP模型下，首先应用层将HTTP数据给传输层，并在外层包上TCP首部信息，到网络层则包装上IP首部信息，到链路层则一般包上以太网的首部信息。

4.客户端和服务端的传输层会发起一条TCP链接来传输网络包，TCP/IP经3次握手便可传输信息。

5.服务端则以相同的方式发送数据回客户端，客户端收到数据解封装，然后浏览器内核渲染界面。

## 8.TCP协议的三次握手，以及四次挥手的过程与必要性

开始时，客户端处于Closed状态，服务端处于 Listen状态。
* 第一次握手：客户端给服务端发送SYN报文，并指定初始化序列号ISN（C）。首部同步位SYN = 1，初始序列号seq=x。
* 二次握手：服务端接收到SYN报文，会以自己的SYN报文作为应答，并指定自己的初始化序列号ISN（S），同时设ACK为ISN（C） + 1，用于表明自己接收到了客户端的SYN。
* 三次握手：客户端收到 SYN 报文之后，会发送一个 ACK 报文，当然，也是一样把服务器的 ISN + 1 作为 ACK 的值，表示已经收到了服务端的 SYN 报文，此时客户端处于 ESTABLISHED 状态。服务器收到 ACK 报文之后，也处于 ESTABLISHED 状态，此时，双方已建立起了连接。
* 四次挥手：

确认报文段ACK=1，确认号ack=y+1，序号seq=x+1（初始为seq=x，第二个报文段所以要+1），ACK报文段可以携带数据，不携带数据则不消耗序号。

发送第一个SYN的一端将执行主动打开（active open），接收这个SYN并发回下一个SYN的另一端执行被动打开（passive open）。



## 9.如何解决以及排查Linux服务器出现的大量CLOSE_WAIT问题？简述CLOSE_WAIT出现原因？

## 10.什么是线程和进程，有什么区别？

## 11.线程间的常用通信方法？

## 12.有3个线程，打印1到100的素数，要求线程1，2，3顺序打印素数

## 13.ORM是什么？有什么用？